In [53]:
import datasets
import uuid

d = datasets.load_dataset("AI-MO/wholeproof-moon-santa-prover-v1-1M-241231", split="train", cache_dir="/mnt/moonfs/kimina-m2/.cache/ttmmpp")
samples = []


d = d.select(range(10000))
for sample in d:
    sample["proof_id"] = str(uuid.uuid4())
    samples.append(sample)


In [54]:
print(len(samples))
for sample in d:
    sample["ori_proof"] = sample["proof"]
    sample["proof"] = sample["proof_input"][:-1] + " sorry\n"


10000


In [55]:
import os

os.environ["AUTOFORMALIZER_WORKSPACE"] = "/home/wanghaiming/project"

from autoformalizer.clients.lean4_client import Lean4Client, batch_verify_proof



client = Lean4Client(
    "http://lean4-evaluator.app.msh.team/",
)

results = batch_verify_proof(
    client,
    samples,
    batch_size=1,
    num_threads=250,
    infotree_type="full"
)


2024-12-31 11:38:48.992 | INFO     | autoformalizer.clients.lean4_client:batch_verify_proof:562 - Processing 10000 samples in 10000 batches of size 1
2024-12-31 11:38:49.676 | INFO     | autoformalizer.clients.lean4_client:batch_verify_proof:578 - Number of lean server: 5
Verifying proofs: 100%|██████████| 10000/10000 [02:28<00:00, 67.34it/s]


In [65]:
print(results[2712]["lean_feedback"])

{"code": "7f6d2b3a-757d-4088-9d53-d77942b396d5", "error": "Lean process timed out", "response": null}


In [ ]:
import json

def extract_compressed_lemma_and_theorems(trees):
    """
    Extracts theorems and lemmas from the infotree
    """
    statements = []
    for tree in trees:
        try:
            tree_pp = tree["node"]["stx"]["pp"]
            if tree_pp.startswith("theorem") or tree_pp.startswith("lemma") or tree_pp.startswith("example"):
                theorem_text = "".join(tree_pp.split())

                # the proof content will always be the second last child
                child = tree["children"][-2]
                proof_pp = "".join(child["node"]["stx"]["pp"].split())
                assert proof_pp in theorem_text
                theorem_statment = theorem_text[:theorem_text.index(proof_pp)]
                statements.append((theorem_statment, proof_pp, theorem_text))
        except KeyError:
            continue
    return statements

def extract_proof_input_output(statements, code):
    """
    Extracts the proof input and output from the proof code
    """
    proof_input_and_outputs = []
    for statement in statements:
        packed_statement, packed_proof, _ = statement
        if packed_proof.startswith("by"):
            packed_statement += "by"

        # Compress code by removing all whitespace characters
        compressed_code = ''.join(code.split())

        # Find the starting index of B in the compressed A
        index = compressed_code.find(packed_statement)
        if index == -1:
            raise ValueError("Compressed string B not found in A")

        # Determine the end position of the matched B in compressed A
        end_compressed = index + len(packed_statement)

        # Map the end_compressed position back to the original A's index
        compressed_count = 0
        split_point = None
        for i, c in enumerate(code):
            if not c.isspace():
                compressed_count += 1
                if compressed_count == end_compressed:
                    split_point = i + 1  # Include the current character
                    break
                
        # If the split point wasn't found within A, set it to the end of A
        if split_point is None:
            split_point = len(code)

        # Split A into before and after
        proof_input = code[:split_point]
        proof_output = code[split_point:]

        assert proof_input[-1] == packed_statement[-1]
        proof_input_and_outputs.append((proof_input, proof_output))
       
    return proof_input_and_outputs

for idx, r in enumerate(results):
    if r["lean_feedback"] is None:
        print(idx, r)
        continue
    feedback = json.loads(r["lean_feedback"])
    if feedback["response"] is None:
        print(idx, r)
        continue
    trees = feedback["response"].get("infotree", None)
    res = extract_lemma_and_theorems(trees)
    iando = extract_proof_input_output(res, r["proof"])
    for io in iando:
        print("========INPUT============")
        print(io[0])
        print("========OUTPUT============")
        print(io[1])
        print("======================")
        print("\n\n\n\n\n")

In [ ]:
import pickle
full_proof_result = pickle.load(open("results.pkl","rb"))


for result in results:
    if result["is_valid_with_sorry"] is False:
        for r in full_proof_result:
            if r["proof_id"] == result["proof_id"] and r["is_valid_no_sorry"] is True:
                print("====================================")
                print("proof_id: ",result["proof_id"])
                print("source: ",result["source"])
                print(result["proof"])
                print("====================================")
                print(result["ori_proof"])
                print("====================================")
                print("\n\n")
                break


In [30]:
cnt, total = 0, 0
for res in results:
    if res["is_valid_with_sorry"]:
        cnt += 1
    total += 1

print(cnt, total, cnt / total)

49999 50000 0.99998


In [11]:
import pickle
pickle.dump(results, open("results.pkl", "wb"))

In [21]:


from collections import Counter


cnt = 0
source = []
for res in results:
    if res["is_valid_with_sorry"]:
        cnt += 1
    else:
        source.append(res["source"])
print(cnt/len(results))
cc = Counter(source)

all_source = []
for res in results:
    all_source.append(res["source"])

cc_all = Counter(all_source)
print(cc)
print(cc_all)
print(f"{'source':<25} {'total':<8} {'invalid':<8} {'ratio':<5}")
for k, v in cc_all.items():
    print(f"{k:<25} {v:<8} {cc[k]:<8} {cc[k]/v:<5}")

0.6496911488488275
Counter({'lean_workbook': 13474, 'deepseekproverv1': 4566, 'aops-wiki': 23, 'aops-forum': 7, 'aops': 5, 'number-theory-books': 4, 'unknown': 4, 'cnk12': 3, 'olympiads-ref': 2, 'math-train': 2, 'math-test': 1})
Counter({'deepseekproverv1': 27503, 'lean_workbook': 13474, 'olympiads': 5125, 'math-train': 1991, 'math-test': 1334, 'aops': 1150, 'cnk12': 382, 'number-theory-books': 280, 'aops-forum': 164, 'aops-wiki': 122, 'unknown': 95, 'olympiads-ref': 23})
source                    total    invalid  ratio
aops-wiki                 122      23       0.1885245901639344
olympiads-ref             23       2        0.08695652173913043
aops                      1150     5        0.004347826086956522
cnk12                     382      3        0.007853403141361256
aops-forum                164      7        0.042682926829268296
number-theory-books       280      4        0.014285714285714285
unknown                   95       4        0.042105263157894736
math-train           

In [4]:
import datasets

data = datasets.load_from_disk("/mnt/moonfs/kimina-m2/.cache/wholeproof-santa-data-no-negation-241230")

Loading dataset from disk:   0%|          | 0/320 [00:00<?, ?it/s]

In [5]:
# random select 1M samples
import random
random.seed(42)
sub_data = data.select(random.sample(range(len(data)), 1_000_000))

In [6]:
sub_data.push_to_hub("AI-MO/wholeproof-moon-santa-prover-v1-1M-no-negation-241231", private=True)

Uploading the dataset shards:   0%|          | 0/6 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/167 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/167 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/167 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/167 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/167 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/167 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/667 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AI-MO/wholeproof-moon-santa-prover-v1-1M-no-negation-241231/commit/9483956368f6d268f5e154d7c48fd00297ba06ab', commit_message='Upload dataset', commit_description='', oid='9483956368f6d268f5e154d7c48fd00297ba06ab', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/AI-MO/wholeproof-moon-santa-prover-v1-1M-no-negation-241231', endpoint='https://huggingface.co', repo_type='dataset', repo_id='AI-MO/wholeproof-moon-santa-prover-v1-1M-no-negation-241231'), pr_revision=None, pr_num=None)

In [6]:
import datasets


d = datasets.load_dataset('AI-MO/wholeproof-test-data-241229', split="train")


AttributeError: module 'dill' has no attribute '_dill'

In [ ]:
lengths = []
for sample in d:
    lengths.append(len(sample["proof_input"]))
    

In [ ]:
for sample in d:
    if len(sample["proof_input"]) > 500:
        print("====================================")
        print(sample["uuid"])
        print(sample["proof_input"])
        print("\n\n\n\n\n")


In [5]:
import hashlib

def hash_to_float(s: str) -> float:
    # Hash the string using SHA-256
    hash_value = hashlib.sha256(s.encode()).hexdigest()
    
    # Convert the hex hash value to an integer
    int_value = int(hash_value, 16)
    
    # Normalize the integer value to the range [0, 1]
    return int_value / (2**256 - 1)

# Example usage:
s = "hellodsfdsfdsdf"
hashed_value = hash_to_float(s)
print(hashed_value)

0.5838946202125024


In [1]:
import datasets

d = datasets.load_dataset("AI-MO/human-statements-moon-new-year-prover-v1-merged", split="train", cache_dir="/mnt/moonfs/kimina-m2/.cache/ttmmpp")

Generating train split:   0%|          | 0/4590 [00:00<?, ? examples/s]

In [2]:
from collections import defaultdict

ori_d = datasets.load_dataset("AI-MO/human-statements-dataset-v1-20250103", split="train", cache_dir="/mnt/moonfs/kimina-m2/.cache/ttmmpp")

catagory = defaultdict(int)
for sample in ori_d:
    if sample["tags"] is not None:
        cata = cata = sample["tags"][-1]
        catagory[cata] += 1
catagory

defaultdict(int,
            {'human-statements-source:inhouse-evaluation-v1-20250102-train': 1578,
             'human-statements-source:PutnamBench-lean4-train': 327,
             'human-statements-source:miniF2F-valid': 243,
             'human-statements-source:proofnet-valid': 169})

In [4]:


pos_result = defaultdict(int)
neg_result = defaultdict(int)
catagory = defaultdict(int)
for sample in d:
    if sample["tags"] is None:
        continue
    cata = sample["tags"][-1]
    if sample["statement_id"].startswith("neg_"):
        if sample["n_correct_proofs"] > 0:
            neg_result[cata] += 1
            # if cata == "human-statements-source:miniF2F-valid":
                # print(sample["statement_id"])
                # print(sample["one_formal_proof"])
    else:
        if sample["n_correct_proofs"] > 0:
            pos_result[cata] += 1
        catagory[cata] += 1

# print positive result
print("positive result")
for k, v in pos_result.items():
    print(k, v, catagory[k], v/catagory[k])

print("negative result")
for k, v in neg_result.items():
    print(k, v, catagory[k], v/catagory[k])

positive result
human-statements-source:inhouse-evaluation-v1-20250102-train 890 1578 0.5640050697084917
human-statements-source:miniF2F-valid 175 243 0.720164609053498
human-statements-source:PutnamBench-lean4-train 160 327 0.4892966360856269
human-statements-source:proofnet-valid 41 169 0.24260355029585798
negative result
human-statements-source:PutnamBench-lean4-train 156 327 0.47706422018348627
human-statements-source:miniF2F-valid 4 243 0.01646090534979424
human-statements-source:proofnet-valid 5 169 0.029585798816568046


In [22]:
import datasets

d = datasets.load_dataset("AI-MO/human-statements-dataset-v1-mathlib-20250106", split="train", cache_dir="/mnt/moonfs/kimina-m2/.cache/ttmmpp")
d[0]["formal_statement"]

"import Mathlib.Combinatorics.SimpleGraph.Finite\nimport Mathlib.Combinatorics.SimpleGraph.Maps\n\nopen Finset\n\nnamespace SimpleGraph\n\nvariable {V : Type*} (G : SimpleGraph V) (s t : V)\n\nnamespace Iso\n\nvariable {G} {W : Type*} {G' : SimpleGraph W} (f : G ≃g G')\n\ninclude f in\ntheorem card_edgeFinset_eq [Fintype G.edgeSet] [Fintype G'.edgeSet] :\n    G.edgeFinset.card = G'.edgeFinset.card := by \n"

In [16]:
from openai import OpenAI
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/mnt/moonfs/kimina-m2/models/deepseekprover/DeepSeek-Prover-V1.5-RL")
formal_statement = d[998]["formal_statement"]
code = f"Complete the following Lean 4 code:\n\n```lean4\n{formal_statement}"
print(len(code[-4000:]))
client = OpenAI(base_url="https://deepseek-ai-DeepSeek-Prover-V1_5-RL.app.msh.team/v1/", api_key="EMPTY")
res = client.completions.create(
    model="deepseek-ai-DeepSeek-Prover-V1_5-RL",
    prompt=code,
    max_tokens=2048,
)
client.token
print(res.choices[0].text)

4000
    rw [mul_comm] at H
    exact H.of_mul_right_left

theorem IsCoprime.mul_left_iff : IsCoprime (x * y) z ↔ IsCoprime x z ∧ IsCoprime y z :=
  ⟨fun H => ⟨H.of_mul_left_left, H.of_mul_left_right⟩, fun ⟨H1, H2⟩ => H1.mul_left H2⟩

theorem IsCoprime.mul_right_iff : IsCoprime x (y * z) ↔ IsCoprime x y ∧ IsCoprime x z := by
  rw [isCoprime_comm, IsCoprime.mul_left_iff, isCoprime_comm, @isCoprime_comm _ _ z]

theorem IsCoprime.of_prod_left (H : IsCoprime (x * y) z) : IsCoprime x z :=
  let ⟨a, b, H⟩ := H
  ⟨a * y, b, by rw [mul_right_comm, mul_assoc, H, mul_one]⟩

theorem IsCoprime.of_prod_right (H : IsCoprime x (y * z)) : IsCoprime x y := by
  rw [isCoprime_comm] at H ⊢
  exact H.of_prod_left

theorem IsCoprime.of_prod_mul (H : IsCoprime (x * y) (x * z)) : IsCoprime x y := by
  rw [isCoprime_comm] at H
  exact H.of_prod_left.of_prod_right

theorem IsCoprime.of_mul_prod (H : IsCoprime x (y * z)) : IsCoprime y z :=
  isCoprime_comm.1 <| H.of_prod_right.of_prod_left

theorem IsCoprime.of

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/mnt/moonfs/kimina-m2/models/deepseekprover/DeepSeek-Prover-V1.5-RL")
for sample in d:
    formal_statement = sample["formal_statement"]
    code = f"Complete the following Lean 4 code:\n\n```lean4\n{formal_statement}"[-4000:]
    tl = len(tokenizer.encode(code))
    if tl > 2000:
        print(len(code), tl)


In [2]:
import datasets

data = datasets.load_dataset("AI-MO/auto-statements-moon-santa-prover-v1", split="train", cache_dir="/mnt/moonfs/kimina-m2/.cache/ttmmpp")

Resolving data files:   0%|          | 0/149 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/149 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/148 [00:00<?, ?it/s]

In [3]:
data = data.select(range(1000))

In [4]:
data_df = data.to_pandas()

print("1")
pos_statements = data_df[(data_df["is_negation"] == False) & (data_df["n_correct_proofs"] > 0)]["statement_id"].unique()
neg_statements = data_df[(data_df["is_negation"] == True) & (data_df["n_correct_proofs"] > 0)]["statement_id"].unique()

# filter out statements with n_correct_proofs > 10
# data_df = data_df[data_df["n_correct_proofs"] < 10]
print("2")
pos_statements = pos_statements.tolist()
neg_statements = neg_statements.tolist()

print("3")
pos_statements_mark = []
for statement in pos_statements:
    pos_statements_mark.append(f"neg_{statement}")
neg_statements_mark = []
for statement in neg_statements:
    neg_statements_mark.append(statement[len("neg_"):])
print("4")
remove_statements = set(pos_statements_mark + neg_statements_mark)
data_df = data_df[~data_df["statement_id"].isin(remove_statements)]
print("5")

# filter out statements with n_correct_proofs > 10
data_df = data_df[data_df["n_correct_proofs"] < 10]
print("6")

data_df


1
2
3
4
5
6


,statement_id,natural_language,is_negation,uuid,formal_statement,proof_id,n_proofs,n_correct_proofs,correct_proof_samples,one_formal_proof
0,30b5f8f9-3e02-40a1-9ee0-7f79b5cceef3,"If $2^8=4^x$, what is the value of $x$?\n\nThe...",False,4563f920-7246-5215-9bdd-1e71c0408b9e,import Mathlib\n\ntheorem algebra_18610 (x : ℕ...,"[30b5f8f9-3e02-40a1-9ee0-7f79b5cceef3_0, 30b5f...",32,0,[import Mathlib\n\ntheorem algebra_18610 (x : ...,
1,neg_30b5f8f9-3e02-40a1-9ee0-7f79b5cceef3,"If $2^8=4^x$, what is the value of $x$?\n\nThe...",True,4563f920-7246-5215-9bdd-1e71c0408b9e,import Mathlib\n\ntheorem negated_algebra_1861...,"[neg_30b5f8f9-3e02-40a1-9ee0-7f79b5cceef3_0, n...",10,0,[],
2,3c209234-bc89-4348-befb-d44041ab8ff3,"If $2^8=4^x$, what is the value of $x$?\n\nThe...",False,4563f920-7246-5215-9bdd-1e71c0408b9e,import Mathlib\n\ntheorem algebra_18610 (x : ℕ...,"[3c209234-bc89-4348-befb-d44041ab8ff3_0, 3c209...",32,0,[],
3,neg_3c209234-bc89-4348-befb-d44041ab8ff3,"If $2^8=4^x$, what is the value of $x$?\n\nThe...",True,4563f920-7246-5215-9bdd-1e71c0408b9e,import Mathlib\n\ntheorem negated_algebra_1861...,"[neg_3c209234-bc89-4348-befb-d44041ab8ff3_0, n...",21,0,[],
4,b3e8346d-4938-4b9f-8a9a-545ce6d0201f,"If $2^8=4^x$, what is the value of $x$?\n\nThe...",False,4563f920-7246-5215-9bdd-1e71c0408b9e,import Mathlib\n\ntheorem algebra_18610 (x : ℕ...,"[b3e8346d-4938-4b9f-8a9a-545ce6d0201f_0, b3e83...",32,0,[],
...,...,...,...,...,...,...,...,...,...,...
978,45ed7ff7-b5d3-40b9-a430-83ec8076232a,A bookstore is deciding what price it should c...,False,5eec702b-27ae-51ba-baaa-431893af28ad,import Mathlib\n\ntheorem algebra_18879 {f : ℝ...,"[45ed7ff7-b5d3-40b9-a430-83ec8076232a_0, 45ed7...",32,0,[],
979,neg_45ed7ff7-b5d3-40b9-a430-83ec8076232a,A bookstore is deciding what price it should c...,True,5eec702b-27ae-51ba-baaa-431893af28ad,import Mathlib\n\ntheorem negated_algebra_1887...,"[neg_45ed7ff7-b5d3-40b9-a430-83ec8076232a_0, n...",30,0,[],
980,9d663622-4383-4bd5-be1a-1b9af1fdbef6,A bookstore is deciding what price it should c...,False,5eec702b-27ae-51ba-baaa-431893af28ad,import Mathlib\n\ntheorem algebra_18879 {p : ℝ...,"[9d663622-4383-4bd5-be1a-1b9af1fdbef6_0, 9d663...",13,6,[import Mathlib\n\ntheorem algebra_18879 {p : ...,import Mathlib\n\ntheorem algebra_18879 {p : ℝ...
990,9edba20e-b435-4b84-93d1-f21ceba27a7c,A bookstore is deciding what price it should c...,False,5eec702b-27ae-51ba-baaa-431893af28ad,import Mathlib\n\ntheorem algebra_18879 {p : ℝ...,"[9edba20e-b435-4b84-93d1-f21ceba27a7c_0, 9edba...",17,0,[],


In [ ]:
import multiprocessing as mp
import queue

a = mp.Manager().Queue()

try:
    a.get(timeout=1)
except queue.Empty:
    print("her2e")

ModuleNotFoundError: No module named 'Queue'